In [1]:
%%capture
!pip install pymongo

In [2]:
import pymongo
import getpass
import pandas as pd
import csv
import configparser
import os


def __connect_mongo_client(path_config='./config/credentials.ini'):
    try:
        # Set clúster MongoDB
        url_template = "mongodb+srv://<username>:<password>@<cluster>.<instance_url>.mongodb.net/test?retryWrites=true&w=majority"
        config = configparser.ConfigParser()
        config.read(path_config)
        username = config.get('Credentials', 'username')
        password = config.get('Credentials', 'password')
        cluster = config.get('Credentials', 'cluster')
        instance_url = config.get('Credentials', 'instance_url')
        #username = getpass.getpass(prompt="Enter username: ")
        #password = getpass.getpass(prompt="Enter password: ")
        #cluster = input("Enter cluster name: ")
        #instance_url = getpass.getpass(prompt="Enter instance url: ")
        try:
            if not username:
                raise ValueError("Username cannot be empty.")
            url = url_template.replace("<username>", username).replace("<password>", password).replace("<cluster>", cluster).replace("<instance_url>", instance_url)
            mongo_client = pymongo.MongoClient(url, serverSelectionTimeoutMS=3000, maxPoolSize=10)
            mongo_client.server_info()
            print ('OK -- Connected to MongoDB at server ')
            return mongo_client
        except pymongo.errors.ServerSelectionTimeoutError as error:
            print ('ERROR -- Timeout with mongoDB connection: ', str(error) + "\n")
        except pymongo.errors.ConnectionFailure as error:
            print ('ERROR -- Could not connect to MongoDB: ', str(error) + "\n")
        except ValueError as error:
            print('ERROR -- Configuration file error: ', str(error) + "\n")
    except pymongo.errors.ConfigurationError as error:
        print('ERROR -- Failed to resolve MongoDB DNS:', str(error) + "\n")


def __connect_mongo_db(mongo_client):
    # Set DB
    if mongo_client == None:
        print("Empty file 'Credentials.ini'")
        return None
    db_list = mongo_client.list_database_names() # Names databases
    print(db_list)
    try:
        name_db = str(input("Name database >>> "))
        db = mongo_client[name_db]
        if name_db in db_list:
            print(f"Connection to database '{name_db}' successful!")
            return db
        else:
            print("The database '{}' does not exist.".format(name_db))
            response = str(input("Do you want to create it? (y/n) >>> "))
            if response.lower() == "y":
                mongo_client[name_db].command("ping")
                db = mongo_client[name_db]
                print("Database '{}' created and conected successfu!".format(name_db))
                return db
            elif response.lower() == "n":
                print("Database not created.")
                return None
            else:
                print("Database not created.")
                return None
    except Exception as error:
        return None
        print("An error occurred by def __connect_mongodb() :", error)


def __connect_mongodb_collection(db):
    if db == None:
        print("'db' not established")
        return None
    try:
        # Set collection
        collection_list = db.list_collection_names() # Names collections
        print(collection_list)
        name_collection = str(input("Name collection >>> "))
        collection = db[name_collection]
        if name_collection in collection_list:
            print(f"Connection to '{name_collection}' in database '{db}' successful!")
            return collection
        else:
            print(f"Collection '{name_collection}' does not exist in database '{db.name}'.")
            response = input("Do you want to create it? (y/n) >>> ")
            if response.lower() == "y":
                collection = db[name_collection]
                print("Collection '{}' created and conected successfu!".format(name_collection))
                return collection
            elif response.lower() == "n":
                print("Collection not created.")
                return None
            else:
                print("Collection not created.")
                return None
    except Exception as error:
        return None
        print("An error occurred by def __connect_mongodb_collection() :", error)


__csv_path = None # Global variable
def __read_csv_data():
    max_attempts = 3
    num_attempts = 0
    try:
        global __csv_path # Use the global variable __csv_path inside the function
        while True:
            if num_attempts == max_attempts:
                print("\nMaximum number of attempts reached. Exiting function.")
                break
            else:
                print(f"The specified PATH is: '{__csv_path}'")
                response = str(input("Are you sure to continue on this PATH? (y/n)"))
                if response.lower() == "y":
                    break
                elif response.lower() == "n":
                    if __csv_path is None:
                        __csv_path = str(input("Enter the PATH of the .csv file >>> "))
                        if not os.path.isfile(__csv_path):
                            print("The specified PATH does not exist.")
                    else:
                        num_attempts += 1
                        __csv_path = str(input("Enter the correct PATH of the .csv file? >>> "))
                        if os.path.isfile(__csv_path):
                            __csv_path = __csv_path
                            break
                        else:
                            print("The specified PATH does not exist.")
                else:
                    num_attempts += 1
    except Exception as error:
        print("An error occurred in __read_csv_data() :", error)

                
def __insert_data_csv_to_mongo(db, collection):
    try:
        global __csv_path # Use the global variable csv_path inside the function
        df = pd.read_csv(__csv_path)
        data = df.to_dict('records') # Convert DataFrame to dictionary list
        collection.insert_many(data) # Insert data into the collection
        print(f"Data from '{__csv_path}' has been inserted into '{collection}' collection.")
    except pymongo.errors.ConnectionFailure as error:
        print("An error occurred by def insert_csv_to_mongo() :", error)
    except pymongo.errors.CollectionInvalid as error:
        print("An error occurred by def insert_csv_to_mongo() :", error)
    except FileNotFoundError as error:
        print("An error occurred by def insert_csv_to_mongo() :", error)
    except Exception as error:
        print("An error occurred by def insert_csv_to_mongo() :", error)


def __get_data_collection(collection):
    try:
        return list(collection.find())
    except Exception as error:
        print("An error occurred by def __get_data_collection() :", error)


In [3]:
mongo_client = __connect_mongo_client()

ERROR -- Configuration file error:  Username cannot be empty.



In [4]:
db = __connect_mongo_db(mongo_client)

Empty file 'Credentials.ini'


In [5]:
collection =__connect_mongodb_collection(db)

'db' not established


In [6]:
csv_path = __read_csv_data()
__insert_data_csv_to_mongo(db,collection)

The specified PATH is: 'None'
Are you sure to continue on this PATH? (y/n)
The specified PATH is: 'None'
Are you sure to continue on this PATH? (y/n)
The specified PATH is: 'None'
Are you sure to continue on this PATH? (y/n)

Maximum number of attempts reached. Exiting function.
An error occurred by def insert_csv_to_mongo() : Invalid file path or buffer object type: <class 'NoneType'>


In [7]:
data = __get_data_collection(collection)
try:
    print(len(data))
except Exception as error:
    print(error)

An error occurred by def __get_data_collection() : 'NoneType' object has no attribute 'find'
object of type 'NoneType' has no len()


This notebook was made by <a href="https://github.com/caminodelaserpiente/mongo_connect">蛇道</a>